# FLAN-T5 모델을 강화 학습(PPO) 및 효율적인 매개변수 미세 조정(PEFT)으로 미세 조정하여 덜 유해한 요약문 생성하기


이 노트북에서는 FLAN-T5 모델을 미세 조정하여 메타 AI의 혐오 발언 보상 모델을 사용해 덜 유해한 내용을 생성할 것입니다. 보상 모델은 주어진 텍스트에 대해 "혐오 아님" 또는 "혐오"를 예측하는 이진 분류기입니다. 근접 정책 최적화(Proximal Policy Optimization; PPO)을 활용하여 모델의 유해성을 줄이는 미세 조정을 진행할 것입니다.

# ml.m5.2xlarge 인스턴스로 테스트

In [ ]:
%pip install torch==2.0.1 torchdata

%pip install --disable-pip-version-check -q \
    transformers==4.34.1 \
    datasets==2.12.0 \
    accelerate==0.23.0 \
    evaluate==0.4.0 \
    trl==0.7.1 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, GenerationConfig
from datasets import load_dataset
from peft import PeftModel, PeftConfig, LoraConfig, TaskType

# trl: 트렌스포머 강화 학습 라이브러리
from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead
from trl import create_reference_model
from trl.core import LengthSampler

import torch
import evaluate

import numpy as np
import pandas as pd

# tqdm: 루프의 진행 상황을 영리한 진행률 표시기로 보여주는 라이브러리
from tqdm import tqdm
tqdm.pandas()

<a name='2'></a>
## 2 - FLAN-T5 모델을 적재하고, 보상 모델 및 유해성 평가기 준비하기

<a name='2.1'></a>
### 2.1 - 데이터를 적재하고 FLAN-T5 모델을 요약에 대한 지침을 활용하여 미세 조정하기

여기서는 동일한 허깅 페이스 데이터 세트 [DialogSum](https://huggingface.co/datasets/knkarthick/dialogsum)과 사전 학습된 모델 [FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5)를 계속 사용할 것입니다.

In [ ]:
from datasets import load_dataset

model_name="google/flan-t5-base"
huggingface_dataset_name = "knkarthick/dialogsum"

dataset_original = load_dataset(huggingface_dataset_name)

dataset_original

Found cached dataset csv (/root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

다음 단계는 데이터 세트를 전처리합니다. 데이터 세트의 일부만 가져온 후, 특정 길이의 대화만 필터링합니다(예제들이 충분히 길면서 읽기 쉽도록 하기 위함입니다). 그런 다음 각 대화에 지침을 추가하고 프롬프트를 토큰화합니다. `input_ids` 필드에는 토큰 ID를 저장하고, `query` 필드에는 프롬프트의 디코딩된 버전을 저장합니다.

아래 셀에서 모든 단계를 차례로 수행할 수 있지만, 모든 작업을 `build_dataset`이라는 함수로 정리하는 것이 좋은 습관입니다.

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, GenerationConfig

def build_dataset(model_name,
                  dataset_name,
                  input_min_text_length,
                  input_max_text_length):

    # 데이터 세트 적재 (이 실습에서는 "학습" 부분만 필요합니다).
    dataset = load_dataset(dataset_name, split="train")

    # 대화의 길이가 input_min_text_length와 input_max_text_length 사이인 대화만 필터링합니다.
    dataset = dataset.filter(lambda x: len(x["dialogue"]) > input_min_text_length and len(x["dialogue"]) <= input_max_text_length, batched=False)

    # 토크나이저 준비. device_map="auto"를 설정하면 GPU와 CPU 간의 전환이 자동으로 이루어집니다.
    tokenizer = AutoTokenizer.from_pretrained(model_name, device_map="auto")

    def tokenize(sample):

        # 각 대화에 지침을 추가.
        prompt = f"""
Summarize the following conversation.

{sample["dialogue"]}

Summary:
"""
        sample["input_ids"] = tokenizer.encode(prompt)

        # 이 지침은 "query"라고 불리며, 이는 PPO 라이브러리의 필수 사항입니다.
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    # 각 대화를 토큰화
    dataset = dataset.map(tokenize, batched=False)
    dataset.set_format(type="torch")

    # 데이터 세트를 학습용과 테스트용으로 나눕니다.
    dataset_splits = dataset.train_test_split(test_size=0.2, shuffle=False, seed=42)

    return dataset_splits

dataset = build_dataset(model_name=model_name,
                        dataset_name=huggingface_dataset_name,
                        input_min_text_length=200,
                        input_max_text_length=1000)

print(dataset)

Found cached dataset csv (/root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached processed dataset at /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-23525d6e58aeae0c.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-162aeda0160c8b6d.arrow


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'query'],
        num_rows: 8017
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'query'],
        num_rows: 2005
    })
})


이전 실습에서는 요약에 대한 지침으로 PEFT 모델을 미세 조정했습니다. 노트북에서의 학습은 데이터의 일부에서 수행되었고, 이후에 완전히 학습된 PEFT 모델의 체크포인트를 S3에서 다운로드했습니다.

여기에서 동일한 모델 체크포인트를 적재합니다.



In [ ]:
!aws s3 cp --recursive s3://dsoaws/models/peft-dialogue-summary-checkpoint/ ./peft-dialogue-summary-checkpoint-from-s3/

download: s3://dsoaws/models/peft-dialogue-summary-checkpoint/tokenizer_config.json to peft-dialogue-summary-checkpoint-from-s3/tokenizer_config.json
download: s3://dsoaws/models/peft-dialogue-summary-checkpoint/adapter_config.json to peft-dialogue-summary-checkpoint-from-s3/adapter_config.json
download: s3://dsoaws/models/peft-dialogue-summary-checkpoint/special_tokens_map.json to peft-dialogue-summary-checkpoint-from-s3/special_tokens_map.json
download: s3://dsoaws/models/peft-dialogue-summary-checkpoint/adapter_model.bin to peft-dialogue-summary-checkpoint-from-s3/adapter_model.bin
download: s3://dsoaws/models/peft-dialogue-summary-checkpoint/tokenizer.json to peft-dialogue-summary-checkpoint-from-s3/tokenizer.json


모델 항목을 나열하고 크기를 확인합니다(15MB 미만).

In [ ]:
!ls -alh ./peft-dialogue-summary-checkpoint-from-s3/adapter_model.bin

-rw-r--r-- 1 root root 14M Jun 15 23:37 ./peft-dialogue-summary-checkpoint-from-s3/adapter_model.bin


모델 파라미터 수를 추출하는 함수를 준비합니다(이전 실습과 동일합니다).

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"\ntrainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

원본 FLAN-T5 모델에 어댑터를 추가합니다. 이전 실습에서는 추론을 위해 완전히 학습된 어댑터만 추가했으므로 저순위 적응(LoRA) 구성 사항을 전달할 필요가 없었습니다. 이제 PEFT 모델을 구성할 때 `is_trainable=True`를 설정하여 저순위 적응 구성을 전달해야 합니다.

In [ ]:
lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name,
                                              torch_dtype=torch.bfloat16)

peft_model = PeftModel.from_pretrained(model,
                                       './peft-dialogue-summary-checkpoint-from-s3/',
                                       lora_config=lora_config,
                                       torch_dtype=torch.bfloat16,
                                       device_map="auto",
                                       is_trainable=True)

print(f'PEFT model parameters to be updated:\n{print_number_of_trainable_model_parameters(peft_model)}\n')


PEFT model parameters to be updated:

trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.41%



이 실습에서는 강화 학습(RL)을 사용하여 대규모 언어 모델(LLM)을 미세 조정할 준비를 하고 있습니다. 강화 학습에 대해 실습의 다음 섹션에서 간단히 설명하겠지만, 현재 단계에서는 PPO 모델을 준비하고 지침에 따라 미세 조정된 PEFT 모델을 전달하는 것만 필요합니다. PPO는 보상 모델에 대해 강화 학습 정책을 최적화하는 데 사용됩니다.

In [ ]:
ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(peft_model,
                                                               torch_dtype=torch.bfloat16,
                                                               is_trainable=True)

print(f'PPO model parameters to be updated (ValueHead + 769 params):\n{print_number_of_trainable_model_parameters(ppo_model)}\n')
print(ppo_model.v_head)

PPO model parameters to be updated (ValueHead + 769 params):

trainable model parameters: 3539713
all model parameters: 251117569
percentage of trainable model parameters: 1.41%

ValueHead(
  (dropout): Dropout(p=0.1, inplace=False)
  (summary): Linear(in_features=768, out_features=1, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


PPO 동안에는 몇 가지 파라미터만 업데이트됩니다. 특히, `ValueHead`의 파라미터가 업데이트됩니다. 이 클래스 모델에 대한 자세한 정보는 [문서](https://huggingface.co/docs/trl/main/en/models#trl.create_reference_model)에서 확인할 수 있습니다. 학습 가능한 파라미터의 수는 $(n+1)*m$으로 계산할 수 있으며, 여기서 $n$은 입력 단위 수(여기서는 $n=768$)이고 $m$은 출력 단위 수(여기서는 $m=1$)입니다. $+1$ 항은 편향(bias)을 고려합니다.

이제 PPO의 고정된 복사 모델을 만들어야 합니다. 이 모델은 미세 조정되지 않는 참조 모델입니다. 참조 모델은 유해성 제거 전의 대규모 언어 모델을 나타냅니다. 참조 모델의 파라미터는 PPO 학습 동안 업데이트되지 않습니다. 이는 의도된 것입니다.

In [ ]:
ref_model = create_reference_model(ppo_model)

print(f'Reference model parameters to be updated:\n{print_number_of_trainable_model_parameters(ref_model)}\n')

Reference model parameters to be updated:

trainable model parameters: 0
all model parameters: 251117569
percentage of trainable model parameters: 0.00%



모든 준비가 끝났습니다. 이제 보상 모델을 준비할 시간입니다!

<a name='2.2'></a>
### 2.2 - 보상 모델 준비

**강화 학습**은 에이전트가 환경에서 행동을 취해 누적 보상을 최대화하는 것을 목표로 하는 기계 학습의 한 유형입니다. 에이전트의 행동은 **정책**에 의해 정의됩니다. 강화 학습의 목표는 에이전트가 **보상 함수**를 최대화하는 최적 또는 거의 최적의 정책을 학습하는 것입니다.

[이전 섹션](#2.1)에서는 원래 정책이 지침을 학습된 PEFT 모델을 기반으로 했습니다. 이는 유해성 제거 전의 대규모 언어 모델입니다. 이후, 레이블러에게 출력의 유해성에 대해 피드백을 요청할 수 있습니다. 그러나 전체 미세 조정 과정에 레이블러를 사용하는 것은 비용이 많이 들 수 있습니다. 이를 피하기 위한 실용적인 방법은 에이전트가 대화 요약의 유해성을 제거하도록 유도하는 보상 모델을 사용하는 것입니다. 직관적인 접근 방식은 두 개의 클래스(`혐오 아님`과 `혐오`)에 대한 감정 분석을 수행합니다. `혐오 아님` 클래스를 출력으로 얻을 가능성이 높을수록 더 높은 보상을 주는 것입니다.

예를 들어, 전체 미세 조정 과정에 레이블러를 사용하는 것이 비용이 많이 들 수 있음을 언급할 수 있습니다. 이를 피하기 위한 실용적인 방법은 보상 모델을 사용하는 것입니다.

모델을 활용하여 생성된 피드백을 사용합니다.

[메타 AI의 RoBERTa](https://huggingface.co/facebook/roberta-hate-speech-dynabench-r4-target) 기반 유해 언어 탐지 모델을 보상 모델로 사용할 것입니다. 이 모델은 **로짓값**을 출력한 다음 두 개의 클래스인 `혐오 아님`과 `혐오`에 대한 확률을 예측합니다. `혐오 아님`의 로짓값을 긍정적인 보상으로 사용합니다. 그런 다음, 이 보상 값을 사용하여 PPO로 모델을 미세 조정합니다.

Create the instance of the required model class for the RoBERTa model. You also need to load a tokenizer to test the model. Notice that the model label `0` will correspond to the class `nothate` and label `1` to the class `hate`.

RoBERTa 모델의 필요한 모델 클래스 인스턴스를 생성합니다. 모델을 테스트하기 위해 토크나이저를 적재해야 합니다. 모델 레이블 `0`은 `혐오 아님` 클래스에 해당하고, 레이블 `1`은 `혐오` 클래스에 해당합니다.

In [ ]:
toxicity_model_name = "facebook/roberta-hate-speech-dynabench-r4-target"
toxicity_tokenizer = AutoTokenizer.from_pretrained(toxicity_model_name, device_map="auto")
toxicity_model = AutoModelForSequenceClassification.from_pretrained(toxicity_model_name, device_map="auto")
print(toxicity_model.config.id2label)

{0: 'nothate', 1: 'hate'}


유해성이 적은 텍스트를 몇 개 선택하고, 이를 토큰화한 후 모델에 입력합니다. 출력 로짓값, 확률 및 미세 조정에 사용될 보상을 출력합니다.

In [ ]:
non_toxic_text = "You are a great person and i like you."

toxicity_input_ids = toxicity_tokenizer(non_toxic_text, return_tensors="pt").input_ids

logits = toxicity_model(input_ids=toxicity_input_ids).logits
print(f'logits [not hate, hate]: {logits.tolist()[0]}')

# [혐오 아님, 혐오]에 대한 확률 출력
probabilities = logits.softmax(dim=-1).tolist()[0]
print(f'probabilities [not hate, hate]: {probabilities}')

# "혐오 아님"에 대한 로짓값을 가져옵니다 - 이것이 보상입니다!
not_hate_index = 0
nothate_reward = (logits[:, not_hate_index]).tolist()
print(f'reward (value of "not hate" logit): {nothate_reward}')

logits [not hate, hate]: [4.653210639953613, -4.178226470947266]
probabilities [not hate, hate]: [0.9998539686203003, 0.00014604683383367956]
reward (value of "not hate" logit): [4.653210639953613]


유해한 댓글을 보여줍니다. 이는 더 유해성이 강하기 때문에 보상이 낮을 것입니다.

In [ ]:
toxic_text = "You are a terrible person and i hate you."

toxicity_input_ids = toxicity_tokenizer(toxic_text, return_tensors="pt").input_ids

logits = toxicity_model(toxicity_input_ids).logits
print(f'logits [not hate, hate]: {logits.tolist()[0]}')

# [혐오 아님, 혐오]에 대한 확률 출력
probabilities = logits.softmax(dim=-1).tolist()[0]
print(f'probabilities [not hate, hate]: {probabilities}')

# "혐오 아님"에 대한 로짓값을 가져옵니다 - 이것이 보상입니다!
nothate_reward = (logits[:, not_hate_index]).tolist()
print(f'reward (value of "not hate" logit): {nothate_reward}')

logits [not hate, hate]: [-2.0644452571868896, 1.6650457382202148]
probabilities [not hate, hate]: [0.02344231680035591, 0.9765576720237732]
reward (value of "not hate" logit): [-2.0644452571868896]


허깅 페이스 추론 파이프라인을 설정하여 유해성 판독기 보상 모델 코드를 단순화합니다.

In [ ]:
device = 0 if torch.cuda.is_available() else "cpu"

sentiment_pipe = pipeline("sentiment-analysis",
                          model=toxicity_model_name,
                          device=device)
reward_logits_kwargs = {
    "top_k": None,  # 모든 점수를 반환합니다.
    "function_to_apply": "none", # "none"으로 설정하여 원시 로짓값을 검색합니다.
    "batch_size": 16
}

reward_probabilities_kwargs = {
    "top_k": None, # 모든 점수를 반환합니다.
    "function_to_apply": "softmax", # "softmax"로 설정하여 소프트맥스를 적용하고 확률을 검색합니다.
    "batch_size": 16
}

print("Reward model output for non-toxic text:")
print(sentiment_pipe(non_toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(non_toxic_text, **reward_probabilities_kwargs))
print("\nReward model output for toxic text:")
print(sentiment_pipe(toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(toxic_text, **reward_probabilities_kwargs))

Reward model output for non-toxic text:
[{'label': 'nothate', 'score': 4.653210639953613}, {'label': 'hate', 'score': -4.178226470947266}]
[{'label': 'nothate', 'score': 0.9998539686203003}, {'label': 'hate', 'score': 0.00014604683383367956}]

Reward model output for toxic text:
[{'label': 'hate', 'score': 1.6650457382202148}, {'label': 'nothate', 'score': -2.0644452571868896}]
[{'label': 'hate', 'score': 0.9765576720237732}, {'label': 'nothate', 'score': 0.02344231680035591}]


출력은 `혐오 아님`(긍정)와 `혐오`(부정) 클래스의 로짓값입니다. 그러나 PPO는 `혐오 아님` 클래스의 로짓값만을 긍정적인 보상 신호로 사용하여 대규모 언어 모델 출력을 유해성이 낮아지는데 도움을 줍니다.

In [ ]:
print(sentiment_pipe(non_toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(non_toxic_text, **reward_probabilities_kwargs))

[{'label': 'nothate', 'score': 4.653210639953613}, {'label': 'hate', 'score': -4.178226470947266}]
[{'label': 'nothate', 'score': 0.9998539686203003}, {'label': 'hate', 'score': 0.00014604683383367956}]


In [ ]:
print(sentiment_pipe(toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(toxic_text, **reward_probabilities_kwargs))

[{'label': 'hate', 'score': 1.6650457382202148}, {'label': 'nothate', 'score': -2.0644452571868896}]
[{'label': 'hate', 'score': 0.9765576720237732}, {'label': 'nothate', 'score': 0.02344231680035591}]


<a name='2.3'></a>
### 2.3. - 유해성 평가

모델을 미세 조정을 통한 유해성 제거 전후로 평가하려면 [유해성 평가 메트릭](https://huggingface.co/spaces/evaluate-measurement/toxicity)을 설정해야 합니다. **유해성 점수**는 0과 1 사이의 소수점 값으로, 1이 가장 높은 유해성을 의미합니다.


In [ ]:
toxicity_evaluator = evaluate.load("toxicity",
                                    toxicity_model_name,
                                    module_type="measurement",
                                    toxic_label="hate")

[2.2](#2.2) 섹션에서 사용한 문장들에 대한 유해성을 계산해 보십시오. 유해성 점수는 보상 모델에서 직접 반환된 `혐오` 클래스의 확률입니다.



In [ ]:
toxicity_score = toxicity_evaluator.compute(predictions=[
    non_toxic_text
])

print("Toxicity score for non-toxic text:")
print(toxicity_score["toxicity"])

toxicity_score = toxicity_evaluator.compute(predictions=[
    toxic_text
])

print("\nToxicity score for toxic text:")
print(toxicity_score["toxicity"])

Toxicity score for non-toxic text:
[0.00014604683383367956]

Toxicity score for toxic text:
[0.9765576720237732]


이 평가기는 [2.1](#2.1) 섹션에서 준비한 대화의 유해성을 계산하는 데 사용될 수 있습니다. 테스트 데이터 세트(`dataset["test"]`), 그 섹션에서 사용한 동일한 토크나이저, [2.2](#2.2) 섹션에서 준비한 고정된 PEFT 모델, 그리고 독성 평가기를 전달해야 합니다. 필요한 단계를 `evaluate_toxicity` 함수로 래핑하는 것이 편리합니다.

In [ ]:
def evaluate_toxicity(model,
                      toxicity_evaluator,
                      tokenizer,
                      dataset,
                      num_samples):

    max_new_tokens=100

    toxicities = []
    input_texts = []
    for i, sample in tqdm(enumerate(dataset)):
        input_text = sample["query"]

        if i > num_samples:
            break

        input_ids = tokenizer(input_text, return_tensors="pt", padding=True).input_ids

        generation_config = GenerationConfig(max_new_tokens=max_new_tokens,
                                             tok_k=0.0,
                                             top_p=1.0,
                                             do_sample=True)

        response_token_ids = model.generate(input_ids=input_ids,
                                            generation_config=generation_config)

        generated_text = tokenizer.decode(response_token_ids[0], skip_special_tokens=True)

        toxicity_score = toxicity_evaluator.compute(predictions=[(input_text + " " + generated_text)])

        toxicities.extend(toxicity_score["toxicity"])

    # 평균 및 표준 편차는 np를 사용하여 계산.
    mean = np.mean(toxicities)
    std = np.std(toxicities)

    return mean, std

이제 모델의 유해성을 미세 조정을 통한 유해성 제거 전과 후에 계산합니다.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map="auto")

mean_before_detoxification, std_before_detoxification = evaluate_toxicity(model=ref_model,
                                                                          toxicity_evaluator=toxicity_evaluator,
                                                                          tokenizer=tokenizer,
                                                                          dataset=dataset["test"],
                                                                          num_samples=10)

print(f'toxicity [mean, std] before detox: [{mean_before_detoxification}, {std_before_detoxification}]')

11it [00:32,  2.92s/it]

toxicity [mean, std] before detox: [0.03544191488551653, 0.04677325571336985]


<a name='3'></a>
## 3 - 요약문에 대한 유해성을 제거하기 위해 미세 조정 수행
PPO을 사용하여 보상 모델에 대해 강화 학습 정책을 최적화합니다.

<a name='3.1'></a>
### 3.1 - `PPOTrainer` 초기화

 설정 매개변수를 설정합니다. `ppo_model`과 토크나이저를 적재합니다. 또한, `ref_model`의 고정된 버전을 적재합니다. 첫 번째 모델은 최적화되며, 두 번째 모델은 쿨백-라이블러(Kullback-Leibler; KL) 발산을 계산하기 위해 참조 모델로 사용됩니다. 이는 PPO 학습에서 추가 보상 신호로 작용하여 최적화된 모델이 원래 대규모 언어 모델에서 너무 벗어나지 않도록 합니다.



In [ ]:
learning_rate=1.41e-5
max_ppo_epochs=1
mini_batch_size=4
batch_size=16

config = PPOConfig(
    model_name=model_name,
    learning_rate=learning_rate,
    ppo_epochs=max_ppo_epochs,
    mini_batch_size=mini_batch_size,
    batch_size=batch_size
)


def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

# 다음 줄을 주석 해제하여 collator를 테스트할 수 있습니다.
# test_data = [{"key1": "value1", "key2": "value2", "key3": "value3"}]
# print(f'Collator input: {test_data}')
# print(f'Collator output: {collator(test_data)}')

ppo_trainer = PPOTrainer(config=config,
                         model=ppo_model,
                         ref_model=ref_model,
                         tokenizer=tokenizer,
                         dataset=dataset["train"],
                         data_collator=collator)

<a name='3.2'></a>
### 3.2 - 모델 미세 조정

미세 조정 루프는 다음 주요 단계로 구성됩니다.
1. 정책 대규모 언어 모델(PEFT 모델)에서 쿼리 응답을 가져옵니다.
2. 유해 언어 탐지 RoBERTa 모델에서 쿼리와 응답의 감정을 가져옵니다.
3. (쿼리, 응답, 보상) 삼중 항목을 사용하여 PPO로 정책을 최적화합니다.

학습이 진행 중이면 다음과 같은 메트릭이 나타납니다.
* `objective/kl`: KL 발산을 최소화합니다.
* `ppo/returns/mean`: 반환되는 평균을 최대화합니다.
* `ppo/policy/advantages_mean`: 이점을 최대화합니다.

In [ ]:
output_min_length = 100
output_max_length = 400
output_length_sampler = LengthSampler(output_min_length, output_max_length)

generation_kwargs = {
    "min_length": 5,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True
}

reward_kwargs = {
    "top_k": None, # 모든 점수를 반환합니다.
    "function_to_apply": "none", # 소프트맥스 없이 윈시 로짓값을 원한다.
    "batch_size": 16
}

max_ppo_steps = 10

for step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    # max_steps에 도달하면 중지.
    if step >= max_ppo_steps:
        break

    prompt_tensors = batch["input_ids"]

    # FLAN-T5/PEFT 대규모 언어 모델에서 응답을 가져옵니다.
    summary_tensors = []

    for prompt_tensor in prompt_tensors:
        max_new_tokens = output_length_sampler()

        generation_kwargs["max_new_tokens"] = max_new_tokens
        summary = ppo_trainer.generate(prompt_tensor, **generation_kwargs)

        summary_tensors.append(summary.squeeze()[-max_new_tokens:])

    # 이 항목은 "response"라고 불러야 합니다.
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in summary_tensors]

    # 보상 출력 값을 계산.
    query_response_pairs = [q + r for q, r in zip(batch["query"], batch["response"])]
    rewards = sentiment_pipe(query_response_pairs, **reward_kwargs)

    # `혐오 아님` 긍정 클래스의 점수이므로 `혐오 아님` 항목을 사용합니다.
    reward_tensors = [torch.tensor(reward[not_hate_index]["score"]) for reward in rewards]

    # PPO 단계 수행.
    stats = ppo_trainer.step(prompt_tensors, summary_tensors, reward_tensors)
    ppo_trainer.log_stats(stats, batch, reward_tensors)

    print(f'objective/kl: {stats["objective/kl"]}')
    print(f'ppo/returns/mean: {stats["ppo/returns/mean"]}')
    print(f'ppo/policy/advantages_mean: {stats["ppo/policy/advantages_mean"]}')
    print('-'.join('' for x in range(100)))

0it [00:00, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
1it [03:08, 188.35s/it]

objective/kl: 32.73497772216797
ppo/returns/mean: -0.5234659910202026
ppo/policy/advantages_mean: 0.010280068963766098
---------------------------------------------------------------------------------------------------


2it [05:56, 176.55s/it]

objective/kl: 35.283809661865234
ppo/returns/mean: -0.6757277250289917
ppo/policy/advantages_mean: -0.00038906000554561615
---------------------------------------------------------------------------------------------------


3it [08:20, 161.53s/it]

...

objective/kl: 33.626686096191406
ppo/returns/mean: -0.5020487904548645
ppo/policy/advantages_mean: 0.013908829540014267
---------------------------------------------------------------------------------------------------


9it [23:46, 154.95s/it]

objective/kl: 30.373188018798828
ppo/returns/mean: -0.5869743227958679
ppo/policy/advantages_mean: 0.0016096821054816246
---------------------------------------------------------------------------------------------------


10it [26:30, 159.03s/it]

objective/kl: 27.400535583496094
ppo/returns/mean: -0.20342299342155457
ppo/policy/advantages_mean: 0.03667036071419716
---------------------------------------------------------------------------------------------------


<a name='3.3'></a>
### 3.3 - 모델 정량적 평가

디스크에서 PPO/PEFT 모델을 다시 적재하고, 테스트 데이터 세트를 사용하여 강화 학습 기반 미세 조정된 모델의 유해성 점수를 평가합니다.

In [ ]:
mean_after_detoxification, std_after_detoxification = evaluate_toxicity(model=ppo_model,
                                                                        toxicity_evaluator=toxicity_evaluator,
                                                                        tokenizer=tokenizer,
                                                                        dataset=dataset["test"],
                                                                        num_samples=10)
print(f'toxicity [mean, std] after detox: [{mean_after_detoxification}, {std_after_detoxification}]')

11it [00:33,  3.06s/it]

toxicity [mean, std] after detox: [0.02853497596118938, 0.03764561218070588]


그리고 참조 모델(유해성 제거 전)과 미세 조정된 모델(유해성 제거 이후)의 유해성 점수를 비교합니다.

In [ ]:
mean_improvement = (mean_before_detoxification - mean_after_detoxification) / mean_before_detoxification
std_improvement = (std_before_detoxification - std_after_detoxification) / std_before_detoxification

print(f'Percentage improvement of toxicity score after detoxification:')
print(f'mean: {mean_improvement*100:.2f}%')
print(f'std: {std_improvement*100:.2f}%')

Percentage improvement of toxicity score after detoxification:
mean: 19.49%
std: 19.51%


<a name='3.4'></a>
### 3.4 - 모델 정성적 평가

테스트 데이터 세트의 일부 예제를 검토합니다. 유해성 평가기를 사용하여 기존 `ref_model`과 미세 조정된(유해성 제거된) `ppo_model`을 비교할 수 있습니다.


​

In [ ]:
batch_size = 20
compare_results = {}

df_batch = dataset["test"][0:batch_size]

compare_results["query"] = df_batch["query"]
prompt_tensors = df_batch["input_ids"]

summary_tensors_ref = []
summary_tensors = []

# ppo 및 기본 모델에서 응답을 가져옵니다.
for i in tqdm(range(batch_size)):
    gen_len = output_length_sampler()
    generation_kwargs["max_new_tokens"] = gen_len

    summary = ref_model.generate(
        input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device),
        **generation_kwargs
    ).squeeze()[-gen_len:]
    summary_tensors_ref.append(summary)

    summary = ppo_model.generate(
        input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device),
        **generation_kwargs
    ).squeeze()[-gen_len:]
    summary_tensors.append(summary)

# 응답을 디코딩.
compare_results["response_before"] = [tokenizer.decode(summary_tensors_ref[i]) for i in range(batch_size)]
compare_results["response_after"] = [tokenizer.decode(summary_tensors[i]) for i in range(batch_size)]

# 쿼리/응답 쌍의 감정 분석을 유해성 제거 이전과 이후로 수행합니다.
texts_before = [d + s for d, s in zip(compare_results["query"], compare_results["response_before"])]
rewards_before = sentiment_pipe(texts_before, **reward_kwargs)
compare_results["reward_before"] = [reward[not_hate_index]["score"] for reward in rewards_before]

texts_after = [d + s for d, s in zip(compare_results["query"], compare_results["response_after"])]
rewards_after = sentiment_pipe(texts_after, **reward_kwargs)
compare_results["reward_after"] = [reward[not_hate_index]["score"] for reward in rewards_after]

100%|██████████| 20/20 [01:49<00:00,  5.47s/it]


결과를 데이터 프레임(DataFrame)으로 저장하고 검토합니다.

In [ ]:
pd.set_option('display.max_colwidth', 500)
df_compare_results = pd.DataFrame(compare_results)
df_compare_results["reward_diff"] = df_compare_results['reward_after'] - df_compare_results['reward_before']
df_compare_results_sorted = df_compare_results.sort_values(by=['reward_diff'], ascending=False).reset_index(drop=True)
df_compare_results_sorted

,query,response_before,response_after,reward_before,reward_after,reward_diff
0,"Summarize the following conversation. #Person1#: Mom, I just finished my paper. Can you proofread it before I hand it in? #Person2#: Sure, let's take a look. Sweetie, this is terrific. Your ideas are so original. #Person1#: Thanks. #Person2#: I can tell you worked hard on it. #Person1#: I really did! I started thinking about what I wanted to say three weeks ago. #Person2#: Well, it was definitely worth all the time. #Person1#: Let's just hope my teacher agrees. Summary: </s>",<pad> #Person1# finishes writing and sends her a paper for proofread. #Person2# says #Person1# worked hard of #Person1#'s paper and says it will be worth it.</s>,"<pad> In proofreading, #Person1# finished #Person1#'s paper. When mom picks it up she is proud of it. The paper is great.</s>",2.428548,3.231158,0.802610
1,"Summarize the following conversation. #Person1#: Oh, my God! What's this? #Person2#: What? #Person1#: Look! This window is open. #Person2#: Did you open it before we left? #Person1#: Are you kidding? It's winter. Why would I open it? #Person2#: I don't know. Wait. Is this yours? #Person1#: No! Oh, my God! Someone has broken into the house. #Person2#: It looks that way. That's probably why the door wasn't locked when we came in. #Person1#: I locked it when I left though. #Person2#: Yes, but t...",<pad> Allen and #Person2# look at the window but thinks someone has broken into the house in winter. They decide to go looking upstairs until one has been found.</s>,<pad> Allen finds a broken door because someone broke in during the winter. #Person1# and Allen aren't sure if someone still lives there.</s>,2.131953,2.552041,0.420088
2,"Summarize the following conversation. #Person1#: Excuse me, could you tell me how to get to the Cross Bakery building? #Person2#: The Cross Bakery building? Oh sure. You're actually walking in the opposite direction. #Person1#: Oh, you're kidding! I thought I was heading east. #Person2#: No, east is the other direction. To get to the Bakery, you need to turn around and go three blocks to Broadway. When you get to the intersection of Broadway and Elm, you hang a left. Go straight down that st...","<pad> #Person1# asks #Person2# how to get to the Cross Bakery building. #Person1# realizes that east North is the other direction. Therefore, #Person2# tells #Person1# that #Person1# will trick #Person1# into moving toward the Cross Bakery on #Person2#'s own safe.</s>",<pad> #Person2# tells #Person1# the directions of getting to the Cross Bakery building. #Person1# realizes #Person1# is wrong and asks #Person2# to show #Person1# the way to the Cross Bakery.</s>,2.267349,2.621099,0.353751
3,"Summarize the following conversation. #Person1#: Could you help me figure out how to look for a job? #Person2#: We have lots of options, what type of job do you need? #Person1#: I want to work in an office. #Person2#: Do you want to work part-time or full-time? #Person1#: I want to work full-time. #Person2#: We have binders with local job listings or you can make use of the computers. OK? #Person1#: I am confused a bit but I am sure that I can figure it out. #Person2#: If you make an appoint...",<pad> #Person1# wants to work in an office. #Person2# tells #Person1# to question the jobs and have an appointment with a job counselor.</s>,<pad> #Person1# asks #Person2# how to look for a job in an office. #Person2# advises him to make an appointment with a job counselor. #Person1# will do good.</s>,1.998148,2.326934,0.328786
4,"Summarize the following conversation. #Person1#: Today more and more families have personal computers. People have wider range of choice to communicate with the outside world. #Person2#: Right. With the establishment of Internet and a lot of web companies, people are getting more and more dependent on the web. #Person1#: One of the common uses of PC is that people can buy goods through it without going out to the physical stores. #Person2#: Can

생성된 시퀀스의 보상 평균/중앙값을 보면 상당한 차이를 관찰할 수 있습니다!